In [ ]:
import re

#### 1-Eliminate implication

In [ ]:
# Initial setup
counter = 97
# Define the replace_char_at_index function
def index_char(str, indx, char_new):
    return str[:indx] + char_new + str[indx + 1:]

# Define the eliminate_implication function
def E_I(expr):
    for i in range(len(expr)):
        if expr[i] == '⇒':
            # Replace '⇒' with '∨'
            expr = index_char(expr, i, "∨")
            for j in range(i-1, -1, -1):
                if expr[j] == '(':
                    # Replace the character before '(' with '~'
                    expr = expr.replace(expr[j-1], "~" + expr[j-1])
                    break
    return expr


# Test the function
logical_expression = "(∀x(P(x) ⇒ Q(x))) & (∀y(Q(y) ⇒ R(y)))"
expression_without_double_negation = E_I(logical_expression)
print("Original expression:", logical_expression)
print("Expression without double negation:", expression_without_double_negation)



####  3-Remove double-not.

In [ ]:
def remove_doube(exp):
  result = []
  i = 0
  while i < len(exp):
    # Check for double negation (~~)
    if exp[i:i+2] == '~~':
      # Skip over the double negation
      i += 2
    else:
      # Append the character to the result
      result.append(exp[i])
      i += 1
  return ''.join(result)

# Example usage
logical_expression = "(~~P(x)=> Q(x))"
expression_without_double_negation = remove_doube(logical_expression)
print("Original expression:", logical_expression)
print("Expression without double negation:", expression_without_double_negation)


#### 2-Move negation inward (Demorgan Law)

In [ ]:
def if_quantis(exp):
    final_expression = ""
    counter = 0
    # Loop through each character in the expression
    while counter < len(exp):
        # check if the current position start with negation forall  or negation there exist 
        if exp.startswith("~∀", counter) or exp.startswith("~∃", counter):
            # Extract the quantifier type (opposite of the negated one)
            quanti = '∃' if exp[counter + 1] == '∀' else '∀'
            # Add the extracted quantifier to the final expression
            final_expression += quanti
            # Move the counter past the negation and quantifier symbols
            counter += 2
            # Add the next character (variable name) and a negation symbol
            variable = exp[counter]
            final_expression += variable + "~"
            counter += 1
        else:
            # If not a negated quantifier, simply add the current character
            final_expression += exp[counter]
            counter += 1
    return final_expression

def MNI(expression):
    if expression.find('~') == 2:
        expression = remove_doube(expression)
    # Apply quantifier manipulation
    expression = if_quantis(expression)
    # Move negation inward using De Morgan's Law
    # Define patterns for negation of AND and OR
    pat_and = r'~\((.*?)\s*&\s*(.*?)\)'
    pat_or = r'~\((.*?)\s*\|\s*(.*?)\)'
     # Apply De Morgan's Law for AND and OR
    expression = re.sub(pat_and, r'(~\1 | ~\2)', expression)
    expression = re.sub(pat_or, r'(~\1 & ~\2)', expression)
    
    # Check if the expression starts with a negation
    if expression.startswith('~'):
        # Remove the negation symbol (~) from the expression
        expression = expression[1:]
        # Negate the entire expression
        expression = "~(" + expression + ")"
    
    # Loop until all negations (~) have been moved inward
    while True:
        # Find the start index of the next negation (~)
        negation_start = expression.find('~(')
        # If no negation is found, exit the loop
        if negation_start == -1:
            break
        
        # Find the corresponding closing parenthesis for the negation
        open_paren_count = 1
        negation_end = negation_start + 2
        for i in range(negation_start + 2, len(expression)):
            if expression[i] == '(':
                open_paren_count += 1
            elif expression[i] == ')':
                open_paren_count -= 1
                if open_paren_count == 0:
                    negation_end = i
                    break
        
        # Extract the inner formula enclosed by the negation
        inner_formula = expression[negation_start + 2:negation_end]
        
        # Split the inner formula by the connective symbols (| or &)
        subExp = inner_formula.split('|') if '|' in inner_formula else inner_formula.split('&')
        
        # Negate each subformula and reconstruct the inner formula
        negated_subformulas = ['~' + subformula if subformula[0] != '~' else subformula[1:] for subformula in subExp]
        # Determine the new logical connective based on the original inner formula
        new_connective = '&' if '|' in inner_formula else '|'
        inner_formula = '(' + new_connective.join(negated_subformulas) + ')'
        
        # Replace the negated expression with the transformed inner formula
        expression = expression[:negation_start] + inner_formula + expression[negation_end + 1:]
    
    # Return the expression with all negations moved inward
    return expression

# Example usage:
formula = "ّّ~~∀x(~(P(x) & Q(x)) | ~R(x))"
formula1 = "~∀x(P(x) & Q(x))"
moved_formula = MNI(formula)
print("Original formula:", formula)
print("Formula with negation moved inward:", moved_formula)
print("=======================================")

####  4-Standardize variable scope

In [ ]:

def SVS(exp):
    result = ""
    for expr in exp:
        new_expr = expr
        vars_seen = set()  # Set to track variables encountered in the expression
        rep_map = {}  # Map old variables to new variables for substitution
        
        # Function to generate a new variable name for substitution
        def var_new(var_old):
            if var_old in rep_map:
                return rep_map[var_old]
            new_var = chr(ord('a') + len(vars_seen))  # Start with 'a' and increment if necessary
            while new_var in vars_seen:
                new_var = chr(ord(new_var) + 1)
            vars_seen.add(new_var)
            rep_map[var_old] = new_var
            return new_var

        counter = 0
        while counter < len(new_expr):
            # Check for universal (∀) or existential (∃) quantifiers
            if new_expr[counter] in "∀∃":
                if counter + 1 < len(new_expr):
                    var = new_expr[counter + 1]
                    if var.islower():  # Ensure it's a lowercase variable
                        # Generate a new variable name for substitution
                        new_var = var_new(var)
                        # Replace occurrences of the old variable with the new variable in the expression
                        new_expr = new_expr[:counter + 1] + new_expr[counter + 1:].replace(var, new_var)
                        counter += len(new_var) - 1
            counter += 1
        result += new_expr   # Concatenate the processed expression with a space
        
    return result.strip()  # Remove trailing space before returning

# Test case

expressions_1 = ['∃x(P(x) & Q(x)) '  ' ∀x(R(x) | S(x))']
print("Original Expressions 1:", expressions_1)
print("SVS Result 1:", SVS(expressions_1))

#### 5- The prenex form (obtained by moving all quantifiers to the left of the
formula.)

In [ ]:
#do
def penex_form(exp):
    quanti_part = ""
    parts = ""
    i = 0
    while i < len(exp):
        if exp[i] == '∀' or exp[i] == '∃':
            quanti_part += exp[i] + exp[i+1] + ' '
            i += 1  # Skip the next character which is the variable
        else:
            parts += exp[i]
        i += 1
    return quanti_part.strip() + " " + parts.strip()

# Test case
expression = '∃x(P(x) ∧ Q(x)) ∧ ∀y(R(y) ∨ S(y))'
print("Original Expression:", expression)
print("Penex Form:", penex_form(expression))


#### 6-Skolemization for existential quantifiers

In [ ]:
def skolemization(exp):
    part = exp.split('∃')
    parts_skol = [part[0]]
    for part in part[1:]:
        if part.strip():
            var, i, exp_in = part.partition('(')
            var += '('  # Adding back the opening parenthesis
            
            # Create Skolem function with parentheses
            func_skol = "f" + "("+var.replace('(', '').replace(')', ')')  + ")" # Add closing parenthesis
            
            # Replace variable x in the inner expression with the Skolem function
            skolemized_inner_exp = exp_in.replace('x',  func_skol )
            
            
            # Append skolemized inner expression to the list of skolemized parts
            parts_skol.append(skolemized_inner_exp)
    return ''.join(parts_skol)

# Example usage:
logical_expression = "∃x (P(x) & Q(x))"
skolemized_expression = skolemization(logical_expression)
print("Original expression:", logical_expression)
print("Skolemized expression:", skolemized_expression)


#### 7-Eliminate universal quantifiers.

In [ ]:
# done
def E_U(exp):
    # Define a pattern to match universal quantifiers and their corresponding variables and inner expressions
    patrn = r'∀(\w+)\s*\((.*?)\)'

    # Use regex to remove all occurrences of universal quantifiers
    withoutForAll = re.sub(patrn, r'(\2)', exp)

    return withoutForAll

# Example usage:
logical_expression = "∀x (P(x) & Q(x))"
expression_without_universal_quantifiers = E_U(logical_expression)
print("Original expression:", logical_expression)
print("Expression without universal quantifiers:", expression_without_universal_quantifiers)


#### 8-Convert to conjunctive normal form

#### - eliminate_bidirectional

In [ ]:
def E_b(exp):
    # Define a pattern to match bidirectional implications
    pat = r'(\([^()]+\)|\b\w+\b(?:\([^()]+\))?)\s*<=>\s*(\([^()]+\)|\b\w+\b(?:\([^()]+\))?)'

    # Repeat until no more bidirectional implications
    while re.search(pat, exp):
        # Find bidirectional implications
        for i in re.finditer(pat, exp):
            left_part, right_part = i.groups()

            # Replace bidirectional implication with conjunction of implications
            exp = exp[:i.start()] + f'({left_part} -> {right_part}) & ({right_part} -> {left_part})' + exp[i.end():]

    return exp

# Test case
test_cases = [
    "(P <=> Q)",
    "(A <=> B) & (B <=> C)",
    "(X <=> Y) & (Y <=> Z)",
    "(A <=> (B <=> C))",
    "(P(x) <=> Q(y)) & (R(z) <=> S(w))"
]

print("Original expressions:")
for test_case in test_cases:
    print(test_case)

print("\nExpressions after bidirectional elimination:")
for test_case in test_cases:
    result = E_b(test_case)
    print(result)



In [ ]:
def to_conj_of_disj(expression):
    # Split the expression by '&' to get individual conjuncts
    conjuncts = expression.split('&')
    
    # Initialize the result as an empty string
    result = ""
    
    # Iterate through each conjunct
    for conjunct in conjuncts:
        # If the conjunct contains '|', split it by '|' to get individual disjunctions
        if '∨' in conjunct:
            disjunctions = conjunct.split('∨')
            # If there are multiple disjunctions, wrap them in parentheses
            if len(disjunctions) > 1:
                conjunct = '(' + conjunct + ')'
        
        # Append the modified conjunct to the result with '&' in between
        result += conjunct + ' & '
    
    # Remove the trailing '&' and return the result
    return result[:-3]  # Remove the last ' & '

# Test case for to_conj_of_disj function
expression = "((~P(a) ∨ ~Q(a))) & ((~Q(b) ∨ R(b)))"
result = to_conj_of_disj(expression)
print("Original expression:", expression)
print("Expression in conjunctive form:", result)
print("===========================================")


In [ ]:
def convert_to_CNF(expression):
    print("Original Expression:", expression)
    
    # Step 1: Eliminate BIDIRECTIONAL
    expression = E_b(expression)
    print("After Step 1 (Eliminate BIDIRECTIONAL):", expression)
    
    # Step 2: Eliminate Implications
    expression = E_I(expression)
    print("After Step 2 (Eliminate Implications):", expression)
    
    # Step 3: Move Negation Inward
    expression = MNI(expression)
    print("After Step 3 (Move Negation Inward):", expression)
    
    # Step 4: Remove Double Negations
    expression = remove_doube(expression)
    print("After Step 4 (Remove Double Negations):", expression)
    
    # Step 5: Standardize Variable Scope
    expression = [expression]
    
    expression = SVS(expression)
    print("After Step 5 (Standardize Variable Scope):", expression)
    # Step 6: Move Universal Quantifiers Outward
    expression = E_U(expression)
    print("After Step 6 (Move Universal Quantifiers Outward):", expression)
    
    # Step 7: Skolemization
    expression = skolemization(expression)
    print("After Step 7 (Skolemization):", expression)
    
    # Step 8: Convert to CNF
    expression = to_conj_of_disj(expression)
    print("After Step 8 (Convert to CNF):", expression)
    
    return expression


logical_expressions = {
    "Logical Expression 1": "(∀x(P(x) ⇒ Q(x))) & (∀y(Q(y) ⇒ R(y)))"
}

cnf_expressions = {}
for name, expression in logical_expressions.items():
    cnf_expression = convert_to_CNF(expression)
    cnf_expressions[name] = cnf_expression
    print("===============================================")

print("\nSet of CNF Expressions:")
for name, expression in cnf_expressions.items():
    print(name, ":", expression)





#### Step 9: Turn Conjunctions into Clauses

In [ ]:
def turn_into_clauses(expression):
    # Split the expression by '&' to get individual conjuncts
    conj = expression.split('&')
    clause = []
    # Iterate through each conjunct
    for conjunct in conj:
        # If the conjunct contains '|', split it by '|' to get individual disjunctions
        if '∨' in conjunct:
            disjun = conjunct.split('∨')
            # If there are multiple disjunctions, wrap them in parentheses
            if len(disjun) > 1:
                conjunct = '(' + conjunct + ')'
       
        # Append the modified conjunct (without extra parentheses) to the list of clauses
        clause.append(conjunct.strip('()'))
    
    return list(set(clause))  # Convert set to list to ensure uniqueness of clauses

# Test cases
test_cases = [
    "((~P(a) ∨ ~Q(a))  &  (~Q(b) ∨ R(b)))"
]

# Test the function with each test case
for i, test_case in enumerate(test_cases, 1):
    print(f"Test Case {i}: {test_case}")
    result = turn_into_clauses(test_case)
    print("Clauses:")
    for clause in result:
        print(clause)
print()


In [ ]:
def turn_into_clauses(expression):
    # Split the expression by '&' to get individual conjuncts
    conj = expression.split('&')
    clause = []
    # Iterate through each conjunct
    for conjunct in conj:
        # If the conjunct contains '|', split it by '|' to get individual disjunctions
        if '∨' in conjunct:
            disjun = conjunct.split('∨')
            # If there are multiple disjunctions, wrap them in parentheses
            if len(disjun) > 1:
                conjunct = '(' + conjunct + ')'
       
        # Append the modified conjunct to the list of clauses
        clause.append(conjunct.strip())
    
    return list(set(clause))  # Convert set to list to ensure uniqueness of clauses

# Test cases
test_cases = [
    "((~P(a) ∨ ~Q(a))  &  (~Q(b) ∨ R(b))"
]

# Test the function with each test case
for i, test_case in enumerate(test_cases, 1):
    print(f"Test Case {i}: {test_case}")
    result = turn_into_clauses(test_case)
    print("Clauses:")
    for clause in result:
        print(clause)
    print()


##### 10.Rename variables in clauses so that each clause has a unique variable name

In [ ]:
# var_counter = 0
# def rename(clauses):
#     # Initialize a counter to keep track of variable names
    
#     global var_counter
#     # Initialize an empty dictionary to store the mapping of old variable names to new ones
#     variable_mapping = {}
#     # Initialize an empty set to keep track of used variable names
#     used_variable_names = set()

#     # Iterate through all clauses to map variables to new names
#     for clause in clauses:
#         # Find all variables in the clause using regular expression
#         variables = re.findall(r'\b[A-Za-z]+\(\w+\)', clause)
        
#         # Iterate through each variable found in the clause
#         for variable in variables:
#             # Extract the function/predicate name and variable name
#             function_name, var_name = re.match(r'(\b[A-Za-z]+)\((\w+)\)', variable).groups()
#             # Check if the variable already exists in the variable mapping
#             if var_name not in variable_mapping:
#                 # If it doesn't exist, create a new variable name and map it
#                 var_counter += 1
#                 mapped_variable = 'var_' + str(var_counter)
#                 # Ensure uniqueness of variable name
#                 while mapped_variable in used_variable_names:
#                     var_counter += 1
#                     mapped_variable = 'var_' + str(var_counter)
#                 variable_mapping[var_name] = mapped_variable
#                 used_variable_names.add(mapped_variable)
    
#     # Initialize an empty set to store the renamed clauses
#     renamed_clauses = set()
    
#     # Iterate through all clauses to replace variables with their new names
#     for clause in clauses:
#         # Replace variables with their mapped names
#         renamed_clause = clause
#         for old_var, new_var in variable_mapping.items():
#             renamed_clause = renamed_clause.replace(old_var, new_var)
#         renamed_clauses.add(renamed_clause)
    
#     # Return the set of clauses with renamed variables
#     return renamed_clauses

# # Test the function with some example clauses
# test_clauses = {
#     "Clause 1": {"(((~P(a) ∨ ~Q(a))) ) & ( ((~Q(y) ∨ R(y))))"},
   
# }

# print("Original Clauses:")
# for name, clause_set in test_clauses.items():
#     print(name + ":")
#     for clause in clause_set:
#         print(clause)
#     print()

# # Rename variables in the clauses
# renamed_clauses = {}
# for name, clause_set in test_clauses.items():
#     renamed_clause_set = rename(clause_set)
#     renamed_clauses[name] = renamed_clause_set

# print("Clauses with Renamed Variables:")
# for name, clause_set in renamed_clauses.items():
#     print(name + ":")
#     for clause in clause_set:
#         print(clause)
#     print()


#### Resolution function 

In [ ]:
# Apply Resolution Procedure
def resolution_procedure(expression):
    # Apply each step of resolution procedure
    expression = E_I(expression)
    expression = MNI(expression)
    expression = remove_doube(expression)
    expression = SVS(expression)
    expression = penex_form(expression)
    expression = skolemization(expression)
    expression = E_U(expression)
    expression = convert_to_CNF(expression) 
    expression = turn_into_clauses(expression)
    # expression = rename(expression)
    return expression
# Test case for resolution_procedure function

logical_expression = "(∀x(P(x) ⇒ Q(x))) & (∀y(Q(y) ⇒ R(y)))"

# Apply the resolution procedure
result = resolution_procedure(logical_expression)

# Define the expected output after applying all steps
expected_result = "[(~P(a) ∨ ~Q(a)) ,   ((~Q(b) ∨ R(b))]"

# Compare the result with the expected output
if result == expected_result:
    print("Test Passed!")
else:
    print("Test Failed!")
    print("Expected Result:", expected_result)
    print("Actual Result:", result)
